In [1]:
data_folder = r'E:\Seminar\Dataset'
data_yaml_path = r'E:\Seminar\data_sub.yaml'
sample_rate = 1

'''
0: player 
1: goalkeeper
2: referee
3: ball
'''

In [3]:
import os
import shutil
import random
import json
import cv2
import yaml

# Data sampling

In [4]:
def create_sampled_data(original_data_dir, new_data_dir, sample_rate):
    # Tạo thư mục mới nếu chưa tồn tại
    os.makedirs(new_data_dir, exist_ok=True)
    
    # Lặp qua các folder train, test, valid
    for folder in ['train', 'test', 'valid']:
        original_folder_path = os.path.join(original_data_dir, folder)
        new_folder_path = os.path.join(new_data_dir, folder)
        os.makedirs(new_folder_path, exist_ok=True)
        
        # Lặp qua các folder SNGS-xxx
        for sngs_folder in os.listdir(original_folder_path):
            sngs_folder_path = os.path.join(original_folder_path, sngs_folder)
            if os.path.isdir(sngs_folder_path):
                new_sngs_folder_path = os.path.join(new_folder_path, sngs_folder)
                os.makedirs(new_sngs_folder_path, exist_ok=True)

                # Xử lý folder img1
                images_folder = os.path.join(sngs_folder_path, 'img1')
                sampled_images_folder = os.path.join(new_sngs_folder_path, 'img1')
                
                os.makedirs(sampled_images_folder, exist_ok=True)

                # Lấy danh sách tất cả các ảnh
                all_images = sorted(os.listdir(images_folder))

                # Chọn n bức ảnh mỗi giây
                num_frames = len(all_images)
                n = sample_rate  # Số bức ảnh lấy mỗi giây
                frame_indices = [i for i in range(0, num_frames, 25)]  # Mỗi giây có 25 frame

                for index in frame_indices:
                    # Lấy n bức ảnh từ mỗi giây
                    for i in range(n):
                        if index + i < num_frames:
                            image_file = all_images[index + i]
                            
                            # Sao chép ảnh vào thư mục mới
                            shutil.copy(os.path.join(images_folder, image_file), sampled_images_folder)

                # Sao chép file Labels-GameState.json nếu có
                json_file_path = os.path.join(sngs_folder_path, 'Labels-GameState.json')
                if os.path.exists(json_file_path):
                    shutil.copy(json_file_path, new_sngs_folder_path)


def create_unique_folder(base_name, base_dir):
    """Tạo một thư mục duy nhất với tên dựa trên base_name."""
    counter = 0
    new_folder_name = base_name
    while True:
        if counter > 0:
            new_folder_name = f"{base_name}_{counter}"
        new_folder_path = os.path.join(base_dir, new_folder_name)
        if not os.path.exists(new_folder_path):
            os.makedirs(new_folder_path)
            return new_folder_path
        counter += 1

# Đường dẫn đến thư mục gốc chứa dữ liệu
original_data_directory = data_folder
# Tạo tên cho thư mục mới 
base_new_folder_name = f"Data_sampling_{os.path.basename(original_data_directory)}"
# Đường dẫn đến thư mục mới sẽ tạo ra
new_data_directory = create_unique_folder(base_new_folder_name, r"E:\Seminar")

create_sampled_data(original_data_directory, new_data_directory, sample_rate)

data_folder = new_data_directory

print("Data sampling done")

Data sampling done


# Gộp ảnh và json files của từng tập

In [5]:
def move_and_rename_files(src_folder, dst_images_folder, dst_json_folder, folder_type):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(dst_images_folder, exist_ok=True)
    os.makedirs(dst_json_folder, exist_ok=True)

    # Xác định tiền tố `a` dựa trên folder_type
    if folder_type == 'train':
        a = '1'
    elif folder_type == 'valid':
        a = '2'
    elif folder_type == 'test':
        a = '3'
    else:
        print(f"Thư mục {folder_type} không hợp lệ.")
        return

    # Duyệt qua các folder con trong src_folder (các folder SNGS-xxx)
    for dir_name in os.listdir(src_folder):
        if dir_name.startswith('SNGS-'):  # Kiểm tra xem folder có phải SNGS-xxx không
            img1_folder = os.path.join(src_folder, dir_name, 'img1')
            json_file_path = os.path.join(src_folder, dir_name, 'Labels-GameState.json')  # Đường dẫn file JSON

            # Lấy 3 số cuối của folder
            suffix = dir_name[-3:]  # 3 số cuối của tên folder SNGS-xxx

            # Di chuyển và đổi tên ảnh
            if os.path.exists(img1_folder):
                for img_file in os.listdir(img1_folder):
                    if img_file.endswith(('.jpg', '.png')):  # Chỉ xử lý các file ảnh
                        src_img_path = os.path.join(img1_folder, img_file)
                        
                        # Đổi tên file ảnh theo định dạng axxximg_name.ext
                        new_img_name = f"{a}{suffix}{img_file}"  # Đổi tên theo yêu cầu
                        dst_img_path = os.path.join(dst_images_folder, new_img_name)

                        # Di chuyển ảnh
                        shutil.move(src_img_path, dst_img_path)
                        #print(f"Đã di chuyển và đổi tên ảnh: {dst_img_path}")

            # Di chuyển file JSON với tên cố định
            if os.path.exists(json_file_path):
                dst_json_path = os.path.join(dst_json_folder, f"{dir_name}_Labels-GameState.json")  # Đặt tên file JSON theo tên folder

                # Đổi tên nếu trùng
                count = 1
                base_name, ext = os.path.splitext(dst_json_path)
                while os.path.exists(dst_json_path):
                    dst_json_path = f"{base_name}_{count}{ext}"
                    count += 1

                # Di chuyển file JSON
                shutil.move(json_file_path, dst_json_path)
                #print(f"Đã di chuyển file JSON: {dst_json_path}")

            # Xóa thư mục SNGS-xxx sau khi đã di chuyển xong
            shutil.rmtree(os.path.join(src_folder, dir_name))  # Xóa thư mục SNGS
            print(f"Đã xóa thư mục: {dir_name}")

# Đường dẫn đến các thư mục train, test, valid
base_sampling_dir = data_folder

train_folder = f"{base_sampling_dir}\\train"
test_folder = f"{base_sampling_dir}\\test"
valid_folder = f"{base_sampling_dir}\\valid"

# Tạo các thư mục đích trong train, test, valid
train_images_folder = os.path.join(train_folder, 'images')
train_json_folder = os.path.join(train_folder, 'json_files')

test_images_folder = os.path.join(test_folder, 'images')
test_json_folder = os.path.join(test_folder, 'json_files')

valid_images_folder = os.path.join(valid_folder, 'images')
valid_json_folder = os.path.join(valid_folder, 'json_files')

# Di chuyển và đổi tên file cho từng thư mục
move_and_rename_files(train_folder, train_images_folder, train_json_folder, 'train')
move_and_rename_files(test_folder, test_images_folder, test_json_folder, 'test')
move_and_rename_files(valid_folder, valid_images_folder, valid_json_folder, 'valid')

Đã xóa thư mục: SNGS-060
Đã xóa thư mục: SNGS-061
Đã xóa thư mục: SNGS-062
Đã xóa thư mục: SNGS-063
Đã xóa thư mục: SNGS-064
Đã xóa thư mục: SNGS-065
Đã xóa thư mục: SNGS-066
Đã xóa thư mục: SNGS-067
Đã xóa thư mục: SNGS-068
Đã xóa thư mục: SNGS-069
Đã xóa thư mục: SNGS-070
Đã xóa thư mục: SNGS-071
Đã xóa thư mục: SNGS-072
Đã xóa thư mục: SNGS-073
Đã xóa thư mục: SNGS-074
Đã xóa thư mục: SNGS-075
Đã xóa thư mục: SNGS-076
Đã xóa thư mục: SNGS-077
Đã xóa thư mục: SNGS-097
Đã xóa thư mục: SNGS-098
Đã xóa thư mục: SNGS-099
Đã xóa thư mục: SNGS-100
Đã xóa thư mục: SNGS-101
Đã xóa thư mục: SNGS-102
Đã xóa thư mục: SNGS-103
Đã xóa thư mục: SNGS-104
Đã xóa thư mục: SNGS-105
Đã xóa thư mục: SNGS-106
Đã xóa thư mục: SNGS-107
Đã xóa thư mục: SNGS-108
Đã xóa thư mục: SNGS-109
Đã xóa thư mục: SNGS-110
Đã xóa thư mục: SNGS-111
Đã xóa thư mục: SNGS-112
Đã xóa thư mục: SNGS-113
Đã xóa thư mục: SNGS-114
Đã xóa thư mục: SNGS-115
Đã xóa thư mục: SNGS-151
Đã xóa thư mục: SNGS-152
Đã xóa thư mục: SNGS-153


# Tạo labels từ file json

In [1]:
data_folder = r'E:\Seminar\Data_sampling_Dataset_3'

In [13]:
def create_labels_from_json(json_folder, images_dir, output_labels_dir):
    # Tạo thư mục để lưu file TXT nếu chưa tồn tại
    os.makedirs(output_labels_dir, exist_ok=True)

    # Giả sử bạn đã biết kích thước ảnh (có thể thay đổi giá trị cho phù hợp)
    image_width = 1920  # Thay đổi nếu kích thước ảnh thực tế khác
    image_height = 1080  # Thay đổi nếu kích thước ảnh thực tế khác

    # Duyệt qua tất cả các file JSON trong thư mục json_files
    for json_file_name in os.listdir(json_folder):
        if json_file_name.endswith('.json'):
            json_file_path = os.path.join(json_folder, json_file_name)

            # Đọc file JSON
            with open(json_file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Kiểm tra sự tồn tại của khóa 'annotations' trong file JSON
            if 'annotations' in data:
                # Duyệt qua tất cả các annotations để ghi vào file TXT
                for annotation in data['annotations']:
                    if 'attributes' in annotation: 
                        team = annotation['attributes']['team'] 
                    # Lấy image_id từ annotation
                    image_id = annotation['image_id']
                    # Tạo tên file TXT tương ứng với image_id
                    txt_file_path = os.path.join(output_labels_dir, f"{image_id}.txt")
                    # Kiểm tra xem file đã tồn tại chưa
                    if not os.path.exists(txt_file_path):
                        # Mở file TXT để ghi
                        with open(txt_file_path, 'w') as txt_file:
                            # Ghi thông tin cho annotation này
                            category_id = annotation['category_id'] - 1  # YOLO yêu cầu class_id bắt đầu từ 0

                            # Kiểm tra sự tồn tại của khóa 'bbox_image' trong annotation
                            if 'bbox_image' in annotation:
                                bbox = annotation['bbox_image']
                                x_center = bbox['x_center']
                                y_center = bbox['y_center']
                                width = bbox['w']
                                height = bbox['h']

                                # Tính toán tọa độ chuẩn hóa
                                x_center_norm = x_center / image_width
                                y_center_norm = y_center / image_height
                                width_norm = width / image_width
                                height_norm = height / image_height

                                # Ghi thông tin vào file TXT
                                txt_file.write(f"{category_id} {x_center_norm} {y_center_norm} {width_norm} {height_norm} {team}\n")
                    else:
                        # Nếu file đã tồn tại, mở file để ghi thêm thông tin
                        with open(txt_file_path, 'a') as txt_file:
                            category_id = annotation['category_id'] - 1  # YOLO yêu cầu class_id bắt đầu từ 0

                            # Kiểm tra sự tồn tại của khóa 'bbox_image' trong annotation
                            if 'bbox_image' in annotation:
                                bbox = annotation['bbox_image']
                                x_center = bbox['x_center']
                                y_center = bbox['y_center']
                                width = bbox['w']
                                height = bbox['h']

                                # Tính toán tọa độ chuẩn hóa
                                x_center_norm = x_center / image_width
                                y_center_norm = y_center / image_height
                                width_norm = width / image_width
                                height_norm = height / image_height

                                # Ghi thông tin vào file TXT
                                txt_file.write(f"{category_id} {x_center_norm} {y_center_norm} {width_norm} {height_norm} {team}\n")
            else:
                print(f"File {json_file_name} không chứa khóa 'annotations'.")

    print(f"Đã chuyển đổi tất cả các annotations sang định dạng YOLO trong folder {output_labels_dir} thành công!")

# Đường dẫn đến các thư mục json_files và images cho train, test, valid
base_dir = data_folder

folders = {
    'train': {
        'json_folder': f'{base_dir}\\train\\json_files',
        'images_dir': f'{base_dir}\\train\\images',
        'output_labels_dir': f'{base_dir}\\train\\labels'
    },
    'test': {
        'json_folder': f'{base_dir}\\test\\json_files',
        'images_dir': f'{base_dir}\\test\\images',
        'output_labels_dir': f'{base_dir}\\test\\labels'
    },
    'valid': {
        'json_folder': f'{base_dir}\\valid\\json_files',
        'images_dir': f'{base_dir}\\valid\\images',
        'output_labels_dir': f'{base_dir}\\valid\\labels'
    }
}

# Gọi hàm cho từng folder
for folder in folders:
    create_labels_from_json(
        folders[folder]['json_folder'],
        folders[folder]['images_dir'],
        folders[folder]['output_labels_dir']
    )

Đã chuyển đổi tất cả các annotations sang định dạng YOLO trong folder E:\Seminar\Data_sampling_Dataset_3\train\labels thành công!
Đã chuyển đổi tất cả các annotations sang định dạng YOLO trong folder E:\Seminar\Data_sampling_Dataset_3\test\labels thành công!
Đã chuyển đổi tất cả các annotations sang định dạng YOLO trong folder E:\Seminar\Data_sampling_Dataset_3\valid\labels thành công!


# Ghép label vào ảnh

In [ ]:
import cv2
import os
import yaml

# Đọc file sub_data.yaml
def read_yaml(yaml_path):
    with open(yaml_path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

def merge_image_with_labels(image_path, labels):
    image = cv2.imread(image_path)

    if image is None:
        return None

    for label in labels:
        # Giả sử mỗi label là một bbox với cấu trúc: x, y, w, h
        x, y, w, h, name = label  # Lấy tọa độ x, y và kích thước w, h

        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Vẽ hình chữ nhật
        cv2.putText(image, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Thêm tên loại lên ảnh

    return image

def process_images(images_folder, yolo_labels_folder, output_folder, yaml_path, n=None):
    os.makedirs(output_folder, exist_ok=True)

    processed_count = 0  # Biến đếm số ảnh đã xử lý

    # Đọc danh sách tên từ file YAML
    yaml_data = read_yaml(yaml_path)
    names = yaml_data['names']  # Lấy danh sách tên loại

    # Lấy danh sách tất cả các file ảnh trong thư mục images
    image_files = [f for f in os.listdir(images_folder) if f.endswith(('.jpg', '.png'))]
    
    # Nếu không cung cấp n, xử lý tất cả các ảnh
    if n is None:
        n = len(image_files)

    # Duyệt qua các file ảnh, nhưng chỉ xử lý tối đa n ảnh
    for image_file in image_files[:n]:
        image_path = os.path.join(images_folder, image_file)
        
        # Tạo tên file TXT tương ứng
        label_file_name = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(yolo_labels_folder, label_file_name)

        # Kiểm tra sự tồn tại của file TXT
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                labels = []
                image = cv2.imread(image_path)  # Đọc ảnh trước để có kích thước ảnh
                if image is None:
                    print(f"Không thể đọc ảnh: {image_path}")
                    continue

                for line in f:
                    # Giả sử mỗi dòng có cấu trúc: class_id x_center y_center width height
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        # Chuyển đổi từ tọa độ center sang bbox
                        class_id = int(parts[0])  # Lấy class_id
                        x_center, y_center, width, height = map(float, parts[1:5])
                        # Chuyển đổi sang tọa độ x, y, w, h
                        x = int((x_center - width / 2) * image.shape[1])
                        y = int((y_center - height / 2) * image.shape[0])
                        w = int(width * image.shape[1])
                        h = int(height * image.shape[0])
                        # Thêm tên loại vào tuple
                        labels.append((x, y, w, h, names[class_id]))  # Thêm tên vào label

            # Gọi hàm merge_image_with_labels để xử lý từng ảnh riêng biệt
            merged_image = merge_image_with_labels(image_path, labels)

            if merged_image is not None:
                output_image_path = os.path.join(output_folder, image_file)
                cv2.imwrite(output_image_path, merged_image)
                #print(f"Đã lưu ảnh đã ghép: {output_image_path}")

                processed_count += 1  # Tăng biến đếm sau khi xử lý ảnh thành công
    print(f"Done {n} images!!!")
                
base_data_sub_dir = data_folder

# Đường dẫn đến thư mục và số lượng ảnh cần xử lý
images_folder = f'{base_data_sub_dir}\\train\\images'
yolo_labels_folder = f'{base_data_sub_dir}\\train\\labels'
output_folder = f'{base_data_sub_dir}\\output_images'
yaml_path = data_yaml_path  # Đường dẫn đến file YAML

# Gọi hàm mà không cung cấp n (sẽ xử lý tất cả ảnh)
process_images(images_folder, yolo_labels_folder, output_folder, yaml_path)

# Gọi hàm với n = 100 (sẽ xử lý 100 ảnh đầu tiên)
#process_images(images_folder, yolo_labels_folder, output_folder, yaml_path, n=100)

# Gộp ảnh thành video để check

In [47]:
import cv2
import os

# Hàm tạo tên mới nếu file đã tồn tại
def get_unique_filename(output_video):
    base, ext = os.path.splitext(output_video)
    count = 1
    new_output_video = output_video

    # Tạo tên mới nếu đã tồn tại file trùng tên
    while os.path.exists(new_output_video):
        new_output_video = f"{base}_{count}{ext}"
        count += 1

    return new_output_video

def images_to_video(folder_path, output_video, n=None, m=0, fps=30, frame_size=None):
    # Tạo thư mục output_videos nếu chưa tồn tại
    output_folder = os.path.dirname(output_video)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Thư mục {output_folder} đã được tạo.")

    # Đổi tên video nếu trùng
    output_video = get_unique_filename(output_video)
    
    # Lấy danh sách các file ảnh jpg trong folder
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".jpg")])
    
    # Nếu không chỉ định n, lấy toàn bộ ảnh từ folder
    if n is None:
        n = len(image_files)

    # Lấy ảnh từ m đến m+n
    selected_images = image_files[m:m+n]
    
    # Kiểm tra nếu danh sách ảnh rỗng
    if len(selected_images) == 0:
        print("Không tìm thấy ảnh nào hoặc phạm vi chọn không hợp lệ.")
        return

    # Lấy kích thước ảnh
    first_image_path = os.path.join(folder_path, selected_images[0])
    frame = cv2.imread(first_image_path)
    if frame is None:
        print(f"Không thể đọc ảnh {selected_images[0]}")
        return
    height, width, layers = frame.shape
    
    # Nếu không chỉ định frame_size, sử dụng kích thước của ảnh đầu tiên
    if frame_size is None:
        frame_size = (width, height)
    
    # Định nghĩa codec và tạo đối tượng VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, frame_size)
    
    if not video_writer.isOpened():
        print("Không thể tạo video_writer.")
        return

    # Ghi từng ảnh vào video
    for image_file in selected_images:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)
        
        # Nếu ảnh không đúng kích thước frame_size, resize nó
        if (image.shape[1], image.shape[0]) != frame_size:
            image = cv2.resize(image, frame_size)

        video_writer.write(image)

    # Giải phóng VideoWriter
    video_writer.release()
    print(f"Video đã được lưu tại: {output_video}")

base_data_sub_dir = data_folder

# Ví dụ sử dụng
folder_path = f"{base_data_sub_dir}\\output_images"  # Đường dẫn tới folder chứa ảnh
output_video = f"{base_data_sub_dir}\\output_videos\\video_output.mp4"  # Đường dẫn file video đầu ra

images_to_video(folder_path, output_video)  # n = 0 thì gộp hết ảnh

#images_to_video(folder_path, output_video, n = 50, m = 0)  # Gộp 100 ảnh từ tấm m 

Thư mục E:\Seminar\Data_sampling_Dataset\output_videos đã được tạo.
Video đã được lưu tại: E:\Seminar\Data_sampling_Dataset\output_videos\video_output.mp4
